In [ ]:
from lib.utils import *
from lib.models import *
from lib.ekyn import *
from lib.env import *
from lib.datasets import *

In [ ]:
class Gandalf(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.encoder = Frodo(n_features=5000,device=DEVICE).to(DEVICE)
        self.lstm = nn.LSTM(16,32,bidirectional=True)
        self.fc1 = nn.Linear(64,3)
    def forward(self,x_2d,classification=True):
        x_2d = x_2d.view(-1,9,1,5000)
        x = torch.Tensor().to(DEVICE)
        for t in range(x_2d.size(1)):
            xi = self.encoder(x_2d[:,t,:,:],classification=False)
            x = torch.cat([x,xi.unsqueeze(0)],dim=0)
        out,_ = self.lstm(x)
        if(classification):
            x = self.fc1(out[-1])
        else:
            x = out[-1]
        return x
model = Gandalf()
model.load_state_dict(torch.load(f'../gandalfs/gandalf_0/best_model.pt',map_location='cpu'))
model.to(DEVICE)
criterion = torch.nn.CrossEntropyLoss()

In [ ]:
FOLD = 14
ids = ['A1','A2','A3','A4','B1','B2','B3','B4','C1','C2','C3','C4','C5','C6','C7','C8','D1','D2','D3','D4','D5','D6']
test_id = ids[FOLD]
print(test_id)
ids.remove(test_id)
subjects = [load_spindle_eeg_label_pair(cohort=id[0],subject=id[1]) for id in [test_id]]
Xs = [subject[0] for subject in subjects]
ys = [subject[1] for subject in subjects]
devloader = DataLoader(dataset=SSDataset(Xs,ys,range(8640)),batch_size=32,shuffle=False)

In [ ]:
loss,report,y_true,y_pred,y_logits = evaluate(devloader,model,criterion,DEVICE)

In [ ]:
confidences = []
idx = y_logits.argmax(axis=1)
for logit,idx in zip(y_logits,idx):
    confidences.append(logit[idx].item())

In [ ]:
fig,axes = plt.subplots(ncols=1,nrows=3,figsize=(10,8),dpi=100,gridspec_kw={'height_ratios': [1,1,2]})
upper = 1000
axes[2].stackplot(torch.arange(len(y_logits[500:upper])),y_logits[500:upper].T)
axes[2].plot(confidences[500:upper],'black')
axes[0].plot(y_true[500:upper])
axes[1].plot(y_pred[500:upper])